In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from modules.helperFunctions import numerator, denominator

def compute_impulse_response(num, den, n_max):
    """
    Input: num - a list of float values representing the coefficients of
                 the numerator of a discrete transfer function
           den - a list of float values representing the coefficients of
                 the denominator of a discrete transfer function
           n_max - integer. Final moment of the simulation of the system response.

    Output:
            n - numpy array of integer time values at which the output is computed
            y - numpy array of float values representing the unit impulse response
                of the system given by the numerator and denominator arguments.
            sys - TransferFunctionDiscrete. System given by num and den.
    """
    sys = signal.dlti(num, den)
    n, y = signal.dimpulse(sys, n=n_max)
    y = y[0].flatten()
    return n, y, sys

def is_stable(poles):
    """
    Inputs: poles - an array of complex values representing the poles of
                    a discrete system
    Output: system_is_stable - boolean value, True if the system is stable, False if
    """
    # TODO
    system_is_stable = all(np.abs(poles) < 1)
    return system_is_stable

def my_conv(x, y):
    """
    Inputs: x, y - 1-D arrays of floats that enter a convolution.
    Output: z - 1-D array of floats. Result of convolution of x and y.
    """

    # Lengths of the input arrays
    len1 = len(x)
    len2 = len(y)

    # Output array to store the convolution result
    result = np.zeros(len1 + len2 - 1)

    # Perform the convolution
    for i in range(len1):
        for j in range(len2):
            result[i + j] += x[i] * y[j]

    return result

# 1. Compute the impulse response h[n] of a system defined by the transfer function
n_max = 6
n, h, system = compute_impulse_response(numerator, denominator, n_max)

# 2. Plot the impulse response h[n]
plt.stem(n, h)
plt.title("Impulse Response h[n]")
plt.xlabel("n")
plt.ylabel("h[n]")
plt.grid()
plt.show()

# 3. Calculate and print system poles and zeros
poles, zeros = system.poles, system.zeros
print("Poles:", poles)
print("Zeros:", zeros)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
circ = plt.Circle((0, 0), radius=1, edgecolor='b', facecolor='None')
plt.grid()
ax.add_patch(circ)
ax.axis('equal')
plt.scatter(poles.real, poles.imag, marker='x', color='r')
plt.xlabel('Re')
plt.ylabel('Im')
plt.title('Poles of system H(z)')
plt.savefig("./cx_out/plot1.png")

# Check system stability
if is_stable(poles):
    print("The system is stable.")
else:
    print("The system is unstable.")

# 4. Simulate the system's response to an input signal x[n]
# Let's create an arbitrary input signal, e.g., a step function
x = np.ones(n_max//2)

# Use convolution to find the full output signal y[n]
my_y = my_conv(x,h)
y = np.convolve(x, h, mode='full')

if (my_y == y).all():
  print('Your convolution is correct!')
else:
  print('Your convolution is NOT correct!')

# 5. Plot input, impulse response, and output signals
fig = plt.figure()
plt.grid()
plt.stem( x, markerfmt='ro', linefmt='r-', basefmt='r-', label="Input x[n]")
plt.stem( h, markerfmt='go', linefmt='g-', basefmt='g-', label="Impulse Response h[n]")
plt.stem(y, markerfmt='bo', linefmt='b-', basefmt='b-', label="Output y[n]")
plt.title("Input, Impulse Response, and Output Signals")
plt.xlabel("n")
plt.ylabel("Value")
plt.legend()
plt.savefig("./cx_out/plot2.png")